# Lab 04: Linear Regression and Causality

This lab asks you to do two tasks:

First, to estimate the effect of Right Hearth
Catheterization (RHC)--inserting a tiny hollow tube along the arterial into
the right side of your heart.  It is often done for diagnostical
purposes (it allows to get different measures right in the heart) and
it's usually considered safe.

We will use a dataset about RHC for critically ill patients and see
if RHC is related to increased death rate.  The dataset is downloaded
from [Vanderbilt
Biostats](http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets) and
more information is available at
[http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html).

Second, you have to create (or re-create) the design matrices of the
first problem, analyze their properties, and compute the regression
coefficients of the first part "manually", i.e. by a simple matrix
expression on computer.


## 1 Get Ready

Load the data.  A tab-separated version is available on canvas (files/data).

1. How many observation and variables do we have? (5735, 63)
2. The most important variables we use below are _death_ (patient
death recorded/not recorded) and
_swang1_ (rhc performed/not performed).  How are these variables coded? These variables are coded as objects

In [38]:
import IPython
print("IPython version:      %6.6s (need at least 1.0)" % IPython.__version__)

# Numpy is a library for working with Arrays
import numpy as np
print("Numpy version:        %6.6s (need at least 1.7.1)" % np.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print("Pandas version:       %6.6s (need at least 0.11.0)" % pd.__version__)

import statsmodels.api as sm
import patsy as pt

file = pd.read_csv("rhc.csv")

IPython version:       6.2.1 (need at least 1.0)
Numpy version:        1.14.0 (need at least 1.7.1)
Pandas version:       0.22.0 (need at least 0.11.0)


## 2 Cross-Sectional Estimator

Estimate, using linear regression, how is the RHC related to mortality.

We start easy: let's just find the relationship between recorded death
and rhc.  The important variables are

* _death_: patient death recorded/not recorded
* _swang1_: rhc performed/not performed

This is called "cross-sectional estimator", we just compare
cross-section of individuals who received and did not receive RHC.

Obviously, this is a very crude measure because the hospitals track
patients over different time periods, and if contact is lost early,
the death may not be recorded.  Another obvious problem is that the
patients have very different medical conditions, a factor that
most likely plays a role in the decision whether to perform RHC.


In [35]:
death = file.death.replace(("Yes", "No"), (1,0))
swang = file.swang1.replace(("RHC", "No RHC"), (1,0))
y = death
X = swang
# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  death   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     2139.
Date:                Wed, 18 Apr 2018   Prob (F-statistic):               0.00
Time:                        11:37:25   Log-Likelihood:                -5989.0
No. Observations:                5735   AIC:                         1.198e+04
Df Residuals:                    5734   BIC:                         1.199e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
swang1         0.6804      0.015     46.245      0.000       0.652       0.709
==============================================================================
Omnibus:                      824.722   Durbin-Watson:                   1.339
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              341.145
Skew:                          -0.409   Prob(JB):                     8.34e-75
Kurtosis:                       2.130   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

r-squared value: 0.272 is low; low correlation between RHC and recorded death

## 3 Address some of the issues

Now let's try to address some of the issues with the estimator above.
The dataset includes many other patient descriptors.


### 3.1 Let's include 'age' and 'sex'.
How are these coded?
age: float
sex: object


What do you find?
age: float sex: object

### 3.2 Include age and sex in the regression

Now allow the death to depend on gender and age, on top of that it may
depend on rhc.  Note that it may not just depend on age in a linear
fashion but in a much more complex way, so include not just $age$ but
also $age^2$ and $age^3$ as explanatory variables.

In [36]:
sex = file.sex.replace(("Female", "Male"), (1,0))
age = file.age

y = death,sex
X = swang, age
# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

/anaconda3/lib/python3.6/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  "samples were given." % int(n), ValueWarning)


ValueError: shapes (2,5735) and (2,5735) not aligned: 5735 (dim 1) != 2 (dim 0)

## 4 Design matrices of the models

Each linear model (and many other models) have associated _design
matrices_.  Design matrix is the matrix of all your explanatory
variables (all x-s) in the final numeric matrix form.  This includes
* adding a constant column
* converting categorical/non-numeric variables into suitable numeric
variables

You next task is to create/extract the design matrices of both of the
models above, investigate their numeric properties (condition
numbers), and solve the linear regression problem in matrix form.

We did not have time in the class to talk about it, but there is a
closed-form solution for the linear regression problem:  beta =
$(X'X)^{-1} X'y$.  Compute this solution and compare with the regression
package output above.


### 4.1 First model

#### 4.1.1 create the design matrix of it, X.

Depending on the way you solved your problem, you may already have
created it.  Depending on the way you solved the problem above, you
may be able to extract it from the existing model.  You may also redo it
manually here.  Remember:
* include the constant term!
* design matrix must be a _matrix_, not data frame or something else.

In [40]:
mat = pt.dmatrix(model, pt.demo_data(X, y))

TypeError: 'in <string>' requires string as left operand, not Series

#### 4.1.2 Compute the condition number of X`X.

You may choose whatever definition you like, but please report what
are you using.

#### 4.1.3 Compute your regression coefficients using the formula above.

Note: you also need your outcome variable $y$ in numeric matrix
form. 

#### 4.1.4 Compare your coefficients here with the OLS results above.